In [97]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [98]:
spark = SparkSession.builder.appName('SMS Confirmations From Users').getOrCreate()

In [99]:
df_fb_sms_sends_input = spark.read.options(header='True', InferSchema='True').csv('fb_sms_sends.csv')
df_fb_confirmers_input = spark.read.options(header='True', InferSchema='True').csv('fb_confirmers.csv')
#df_fb_sms_sends_input.show()
#df_fb_confirmers_input.show()

In [100]:
##filtering out confirmation and friend_req records and  record for date August 4, 2020
df_fr_con = df_fb_sms_sends_input.filter((col("type") != "confirmation") & (col("type") != "friend_request") & (col("ds") == "2020-08-04") )
#df_fr_con.show()

In [101]:
##joining with confirmer table to list out all valid records
df_fr_con = df_fr_con.withColumnRenamed("phone_number","phn_no")
cond = [df_fb_confirmers_input.date == df_fr_con.ds, df_fb_confirmers_input.phone_number == df_fr_con.phn_no]
df_valid_records = df_fb_confirmers_input.join(df_fr_con , cond , 'inner').selectExpr('date','phone_number','type')
#df_valid_records.show()

In [102]:
##calculating percentage of confirmation
cofirmation_percentage = (df_valid_records.count() / df_fr_con.count() * 100)
print(cofirmation_percentage)

20.0
